### Подключение библиотек

In [1]:
#!pip install C:\Users\elena.khotlyannik\Documents\AA\feature-selector
#!pip install hyperopt
#!pip install optuna
#!pip install nlopt

In [2]:
import pyodbc 

import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer as dv

from feature_selector import FeatureSelector

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

import hyperopt
from hyperopt import hp, fmin, tpe
from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK, Trials, SparkTrials

import optuna
import pickle
import nlopt

from scipy.stats import pearsonr

import matplotlib.pyplot as plt
%matplotlib inline

### Определение переменных по умолчанию

In [3]:
# parameters for DATA LOADING:
# 1
# load_table_from - to define what type of loading from data source need to be chosen
# value options: 'db' or 'file'
load_type = 'db'  # values: 'db' or 'file'

# 1.1 group of parameters, which are required if load_table_from = 'db'
pc_driver = 'SQL Server'
pc_server = '.'
pc_database = 'PRICE_FORMATION'
# pc_table   /table with price context data/
pc_table = 'sales_price_context'
# plan_table   /table with plan for product+store data/
plan_table = 'target_product_store_plan'

# 1.2 parameter, which is required if load_table_from = 'file'
# pc_file_data_path /file with price context data /
pc_file_path = 'C:\\Users\\elena.khotlyannik\\Documents\\AA\\milks.csv'
plan_file_path = 'target_product_store_plan.csv'

# 2
# parameters for data preparation

test_proportion = 0.2  #for train/test
target = ['s_count']
#   Определяю признаки, которые не являются предикторами для моделей
special_cols = ['s_date', 'product_id'] + target
#   Определяю категориальные признаки, которые предполагается разложить на  0 1
categorical_cols = ['product_type', 'product_subtype', 'product_category']

test_hyperopt_proportion = 0.2

# 3
# parameters for models
global_var_evals = 32
global_verbose = 0
# 3.1 for GradientBoostingRegressor model (gbr_model)
gbr_param_definition = 'by_optuna' # or 'from_global_vars' or 'load_from_file' or 'by_hyperopt'
# if gbr_param_definition='from_file':
gbr_param_file = 'C:\\Users\\elena.khotlyannik\\Documents\\AA\\gbr_params.csv'
# if gbr_param_definition='from_global_vars':
gbr_max_features = 1
gbr_n_estimators = 1
gbr_learning_rate = 1
gbr_max_depth = 1

# 3.2 for RandomForestRegressor model (rf_model)
rf_param_definition = 'by_optuna'  # 'by_hyperopt' or 'from_global_vars' or 'load_from_file' or 'by_optuna'
# if rf_param_definition='from_file':
rf_param_file = 'C:\\Users\\elena.khotlyannik\\Documents\\AA\\rf_params.csv'
# if rf_param_definition='from_global_vars':
rf_max_features = 2
rf_n_estimators = 2
rf_max_samples = 2
rf_max_depth = 2

# 4.
# parameters for FeatureSelector
fs_cumulative_importance = 0.95
fs_from_file = 'N'
fs_file_path = 'C:\\Users\\elena.khotlyannik\\Documents\\AA\\features_to_remove.csv'
#исключение из листа исключений (список полей, которые не надо исключать из предикторов)
feature_exceptions = ['store_id', 'price', 'year', 'month', 'day']

### Функции загрузки данных

In [4]:
# load_df - function do for data loading
# parameters:
#      load_table_from
#       file_path
#       output_table
#       driver
#       server
#       database


def load_data(load_type=load_type,
              file_path=plan_file_path,
              driver=pc_driver,
              server=pc_server,
              database=pc_database,
              input_table=pc_table):
    result_table = pd.DataFrame
    if load_type == 'db':
        print("Start loading data from " + database + '.' + input_table)
        conn = pyodbc.connect("driver={" + driver + "}; server=" + server + "; database=" + database)
        cur = conn.cursor()
        cur.execute("SELECT * FROM " + input_table)
        cols = [column[0] for column in cur.description]
        input_table1 = input_table
        input_table = cur.fetchall()
        type(input_table)
        cur.close()
        conn.close()
        result_table = pd.DataFrame.from_records(input_table)
        result_table.columns = cols
        print("Finish loading data from " + database + '.' + input_table1)

    elif load_type == 'file':
        print("Start loading data from file " + file_path)
        result_table = pd.read_csv(file_path, sep=',', engine="python", encoding='utf8')
        print("Finish loading data from file " + file_path)

    return result_table


# load_pc - function for price_context loading


def load_pc(load_type=load_type,
            file_path=pc_file_path,
            driver=pc_driver,
            server=pc_server,
            database=pc_database,
            input_table=pc_table):
    result_table = load_data(load_type=load_type,
                             file_path=file_path,
                             driver=driver,
                             server=server,
                             database=database,
                             input_table=input_table)
    return result_table


# load_plan - function for target_plan loading


def load_plan(load_type=load_type,
            file_path=plan_file_path,
            driver=pc_driver,
            server=pc_server,
            database=pc_database,
            input_table=plan_table):
    result_table = load_data(load_type=load_type,
                             file_path=file_path,
                             driver=driver,
                             server=server,
                             database=database,
                             input_table=input_table)
    return result_table


# Загрузка гиперпараметров для модели из файла


def load_params(file_path):
    params = load_data(load_type='file', file_path=file_path)
    best = {}
    for i in range(0, len(params.columns)):
        best.update({params.iloc[0][i]: float(params.iloc[1][i])})

    return best


# Загрузка предикторов, которые надо удалить из датасета для обучения модели. Из файла


def load_features(file_path):
    list_to_remove = load_data(load_type='file', file_path=file_path)
    list_to_remove = list_to_remove.iloc[0].tolist()

    return list_to_remove

### Функции предобработки данных, разделения test/train, X/y

In [5]:
def data_prepare(data_table,
                 special_cols=special_cols,
                 categorical_cols=categorical_cols):

    #   Добавляю ГОД МЕСЯЦ ДЕНЬ из s_date
    price_context.loc[:,'s_date']   = data_table.loc[:, 's_date'].apply(pd.to_datetime)
    price_context.loc[:,'year'] = data_table.loc[:, 's_date'].apply(lambda x : x.year)
    price_context.loc[:,'month'] = data_table.loc[:, 's_date'].apply(lambda x : x.month)
    price_context.loc[:,'day'] = data_table.loc[:, 's_date'].apply(lambda x : x.day)
    
    #отделяю столбцы group_l* , так как они не идуд в обучение. group_l нужно только для пересчета контекста при оптимизации
    groupl_cols = []
    groupl_cols = [col for col in data_table.columns if 'group_l' in col] 
    groupls = data_table[groupl_cols]
    
   
    #   Отделяю special_cols, чтобы потом все собрать (concat)
    spec = data_table[special_cols]       
    
    #   Определяю вещественные признаки, по каторым предполагается заполнить пустые значения нулями
    numeric_cols = list(set(data_table.columns.values.tolist()) - set(categorical_cols) - set(special_cols) - set(groupl_cols))

    #   Для вещественных признаков заменяю null на 0
    zeros = data_table[numeric_cols].fillna(0)
    
    #   Для категориальных признаков  "nan" заменяем на "NA"
    cat=data_table[categorical_cols].astype(str)
    for column in cat.columns:
        cat[column].fillna(value='Na', inplace=True)
        
    # Раскладываю категориальные принаки на  0 1
    from sklearn.feature_extraction import DictVectorizer as DV
    encoder = DV(sparse = False)
    cat2 = encoder.fit_transform(cat.T.to_dict().values())
    cat2=pd.DataFrame(cat2)
    cat2.columns = encoder.feature_names_
    
    # Идею сделать Mean Embanding для категориальных признаков не реализовала
    


    #   Собираю обратно
    data_table_prepared=pd.concat([spec,groupls,zeros,cat2],axis=1) 
    
    return data_table_prepared, groupl_cols

In [6]:
def test_train_split(data_table,
                     test_proportion=test_proportion):
    
    #   получаю список уникальных  дат, отсортированных по возрастанию
    s_date_list = pd.DataFrame(data_table['s_date'].unique()).sort_values(by=0, ascending=True)

    #   определяю количество дат, которые пойдут в test
    test_count = round(len(s_date_list)*test_proportion)

    #   список дат s_date_list отсортирован, поэтому последние  test_count дат идут в s_date_list_test
    s_date_list_test = s_date_list.iloc[-test_count:, :]

    #   min(s_date_list_test[0])   = разделяющая дата. Записи, у которых s_date меньше идут в train, остальное в test
    data_table_train = data_table[data_table['s_date'] < min(s_date_list_test[0])]
    data_table_test = data_table[data_table['s_date'] >= min(s_date_list_test[0])]

    return data_table_train, data_table_test

In [7]:
def x_y_split(data_table,
              target):
    
    column_to_delete = special_cols + groupl_cols

    data_table_x = data_table.drop(column_to_delete, axis=1)
    data_table_y = data_table[target]

    return data_table_x, data_table_y

### Функция ошибки - ОТНОШЕНИЕ СРЕДНИХ WMAPE

In [8]:
def wmape(y_true, y_pred): 
    wmape = 100 * np.sum(np.abs(np.array(y_true) - np.array(y_pred))) /np.sum(np.array(y_true))
   
    return  wmape

### Функция отбора предикторов

In [9]:
# Отбор предикторов по продукту+магазину
# Возвращает лист c предикторами, которые надо удалить, list_to_remove

def feature_selector(x_data=None,
                     y_data=None,
                     cumulative_importance=None,
                     exceptions=feature_exceptions,
                     from_file=None,
                     file_path=None):
    
    if from_file is None or from_file != 'Y':
        from_file = fs_from_file
    
    if from_file == 'Y':
        if file_path is None:
            file_path = fs_file_path
        list_to_remove = load_features(file_path)

    else:
        if cumulative_importance is None:
            cumulative_importance = fs_cumulative_importance
        fs = FeatureSelector(data=x_data, labels=y_data)
        fs.identify_zero_importance(task='regression',
                                    n_iterations=10,
                                    early_stopping=False)
        fs.identify_low_importance(cumulative_importance)
        list_to_remove = fs.record_low_importance.feature.tolist()
        #print('list_to_remove:', list_to_remove)
        #print('exceptions:', exceptions)
        for feature in exceptions:
                if feature in list_to_remove:
                   # print(feature)
                    list_to_remove.remove(feature)
    # return list of features to remove
    return list_to_remove

#  Возвращает датасет с удаленными 
def remove_features(data_table,list_to_remove):
    data_table = data_table.drop(list_to_remove, axis=1, errors='ignore')

    return data_table


#### Функции подбора гиперпараметров gbr_hyperopt, rf_hyperopt,  gbr_optuna , rf_optuna   
###### имеет смысл сделать функции для hyperopt вообще, а потом функции  gbr_hyperopt, rf_hyperopt на ее базе. Так же и optuna.
##### передавать параметры для сетки через атрибуты вызова функции

In [10]:
#   ПОДБОР ПАРАМЕТРОВ для градиентного бустинга HYPEROPT
def gbr_hyperopt(x_train_train, y_train_train, x_train_test, y_train_test, evals):
    
    max_evals = evals
    def objective(space):
        g_model = GradientBoostingRegressor(
            loss='ls',
            random_state=0,
            max_features=space["max_features"],
            n_estimators=space["n_estimators"],
            learning_rate=space["learning_rate"],
            max_depth=space["max_depth"]
        )

        g_model.fit(x_train_train, y_train_train)
        y_predict = np.float64(g_model.predict(x_train_test))
        res = wmape(y_train_test, y_predict)

        return {'loss': res, 'status': STATUS_OK,
                'n_estimators': g_model.n_estimators,
                'learning_rate': g_model.learning_rate,
                'max_depth': g_model.max_depth}

    #   Распределения переменных.
    space = {'n_estimators': scope.int(hp.quniform('n_estimators',  10, 500, 20)),
             'learning_rate': hp.loguniform('learning_rate', -6.9, 2.3),
             'max_depth': scope.int(hp.quniform('max_depth', 2, 10, 2)) ,
             'max_features': scope.int(hp.quniform('max_features', 2, max(4,x_train_train.shape[1]-3),2))
             }
    
  #  spark_trials = pyspark.SparkTrials(parallelism=8)
    #   Поиск лучших параметров GradientBoostingRegressor.
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals
    )

    return best

In [11]:
def rf_hyperopt(x_train_train, y_train_train, x_train_test, y_train_test, evals):

    max_evals = evals

    # Целевая функция.
    def objective(space):
        rf_model = RandomForestRegressor(
            random_state=0,
            n_estimators=space["n_estimators"],
            max_depth=space["max_depth"],
            max_features=space["max_features"]
        )

        rf_model.fit(x_train_train, y_train_train)
        y_predict = np.float64(rf_model.predict(x_train_test))
        res = wmape(y_train_test, y_predict)

        return {'loss': res, 'status': STATUS_OK,
                'n_estimators': rf_model.n_estimators,
                'max_depth': rf_model.max_depth,
                'max_features': rf_model.max_features
                }

    # Распределения переменных.
    space = {
        'n_estimators':  scope.int(hp.quniform('n_estimators', 10, 500, 10)),
        'max_depth':     scope.int(hp.quniform('max_depth', 2, 20, 2)),
        'max_features':  scope.int(hp.quniform('max_features', 2, max(4,x_train_train.shape[1]-3),2))
    }

    # Поиск лучших параметров Random Forest.
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials
    )

    return best

In [12]:
#   ПОДБОР ПАРАМЕТРОВ для градиентного бустинга OPTUNA
def gbr_optuna(x_train_train, y_train_train, x_train_test, y_train_test, evals):

    class Objective:
        def __init__(self):
            global x_train_train, y_train_train, x_train_test, y_train_test

        def __call__(self, trial):
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 10, 500),
                'learning_rate': trial.suggest_uniform('learning_rate', 0.00, 2),
                'max_depth': trial.suggest_int('max_depth', 2, 20) ,
                'max_features': trial.suggest_int('max_features', 3, max(3,x_train_train.shape[1]-3))
                }    

            gbr_model = GradientBoostingRegressor(**params)
            gbr_model.fit(x_train_train, y_train_train)
            y_predict = np.float64(gbr_model.predict(x_train_test))
            res = wmape(y_train_test, y_predict)

            return res 

    # Use Optuna to find the hyperparameters with the highest score    
  #  print('Hyperparameter optimization starting.')
    objective = Objective() #.values.ravel())
    study_gbr = optuna.create_study(direction='minimize', study_name='optuna_GradientBoostingRegressor', load_if_exists=True)
    study_gbr.optimize(objective, n_trials=evals ,n_jobs=-1)
    hist_df_rf = study_gbr.trials_dataframe()

  #  print("Optimized params: {}".format(study_gbr.best_params))

    return study_gbr

In [13]:
#   ПОДБОР ПАРАМЕТРОВ для Random Forest OPTUNA
def rf_optuna(x_train_train, y_train_train, x_train_test, y_train_test, evals):

    class Objective:
        def __init__(self):
            global x_train_train, y_train_train, x_train_test, y_train_test

        def __call__(self, trial):

            params = {
                'n_estimators':  trial.suggest_int('n_estimators', 100, 500),
                'max_depth':     trial.suggest_int('max_depth', 2, 10),
                'max_features':  trial.suggest_int('max_features', 3, max(3,x_train_train.shape[1]-3))
                }    

            rf_model = RandomForestRegressor(**params)
            rf_model.fit(x_train_train, y_train_train)
            y_predict = np.float64(rf_model.predict(x_train_test))
            res = wmape(y_train_test, y_predict)

            return res 

    # Use Optuna to find the hyperparameters with the highest score    
  #  print('Hyperparameter optimization starting.')
    objective = Objective() #.values.ravel())
    study_rf = optuna.create_study(direction='minimize', study_name='optuna_RandomForest', load_if_exists=True)
    study_rf.optimize(objective, n_trials=evals ,n_jobs=-1)
    hist_df_rf = study_rf.trials_dataframe()

   # print("Optimized params: {}".format(study_rf.best_params))

    return study_rf

### Функция создания модели с заданными параметрами

In [14]:
#   Создание модели
def model_formation(X_train_train=None, X_train_test=None, y_train_train=None, y_train_test=None,
                    model_type='gbr',  # 'gbr' / 'rf'  (GradientBoostingRegressor / RandomForestRegressor)
                    param_definition=None,
                    # 'by_hyperopt' / 'by_optuna' / 'from_global_vars' / 'from_file' / none (если none, то параметры для модели надо задать при вызове функции)
                    param_file=None,
                    max_features=None,  # used for gbr and rf
                    n_estimators=None,  # used for gbr and rf
                    learning_rate=None,  # used for gbr
                    max_depth=None,  # used for gbr and rf
                    evals = global_var_evals,
                    verbose = global_verbose
                    ):
    
    best = {}
    if param_definition is None and max_features is None:
        if model_type == 'gbr':
            param_definition = gbr_param_definition
        if model_type == 'rf':
            param_definition = rf_param_definition

    elif param_definition is None and max_features is not None:
        #  Определение гиперпараметров
        best["max_features"] = max_features
        best["n_estimators"] = n_estimators
        best["learning_rate"] = learning_rate
        best["max_depth"] = max_depth


    if param_definition == 'from_file':
        if param_file is None:
            if model_type == 'gbr':
                param_file = gbr_param_file
            if model_type == 'rf':
                param_file = rf_param_file
        best = load_params(param_file)


    elif param_definition == 'by_hyperopt':
        if model_type == 'gbr':
            best = gbr_hyperopt(X_train_train, X_train_test, y_train_train, y_train_test, evals)
        elif model_type == 'rf':
            best = rf_hyperopt(X_train_train, X_train_test, y_train_train, y_train_test, evals)
            
    elif param_definition == 'by_optuna':
        if model_type == 'gbr':
            best = gbr_optuna(X_train_train, X_train_test, y_train_train, y_train_test, evals).best_params
        elif model_type == 'rf':
            best = rf_optuna(X_train_train, X_train_test, y_train_train, y_train_test, evals).best_params

    elif param_definition == 'from_global_vars':
        if model_type == 'gbr':
            best["max_features"] = gbr_max_features
            best["n_estimators"] = gbr_n_estimators
            best["learning_rate"] = gbr_learning_rate
            best["max_depth"] = gbr_max_depth

        elif model_type == 'rf':
            best["max_features"] = rf_max_features
            best["n_estimators"] = rf_n_estimators
            best["max_depth"] = rf_max_depth

    if model_type == 'gbr':
        default_model = GradientBoostingRegressor(
            random_state=0,
            max_features=int(best["max_features"]),
            n_estimators=int(best["n_estimators"]),
            learning_rate=np.float16(best["learning_rate"]),
            max_depth=int(best["max_depth"]),
            verbose=verbose
        )

    elif model_type == 'rf':
        default_model = RandomForestRegressor(
            random_state=0,
            max_features=int(best["max_features"]),
            n_estimators=int(best["n_estimators"]),
            max_depth=int(best["max_depth"]),
            verbose=verbose
        )

   # print(best)

    return default_model


### Функции задания ансамбля

In [15]:
def sr_stacking(list_of_models,models2level):

    level1_estimators = []
  #  print(list_of_models)
  #  print(level1_estimators)
    i = 1
    for model in list_of_models:
        model_name_str = 'model' + str(i)
        i += 1
        level1_estimators.append((model_name_str, model))

 #   print(level1_estimators)
    ensemble_stack = StackingRegressor(
        estimators=level1_estimators,
        final_estimator=models2level,
        passthrough = True,
        n_jobs = -1
    )

    return ensemble_stack

### Функция расчета y_predict, wmape, корреляции пирсона,  также сохранения в словарь

In [16]:
def y_wmape_calc_save(product, store, model, model_tag, X_test, y_test, X_train):
    
    y_predict_test = model.predict(X_test)
    res_test = wmape(y_test, y_predict_test)
    
    dict_for_model[str(product)+str(store)][model_tag] = model
    dict_for_model[str(product)+str(store)][model_tag+'_y_predict'] = y_predict_test
    dict_for_model[str(product)+str(store)]['wmape_'+model_tag+'_test'] = res_test
    
    
    y_predict_test = pd.DataFrame(y_predict_test)
    y_predict_test.columns = ['y_predict']
    product_df_sort = pd.concat([X_test['price'].reset_index(),y_test.reset_index(),y_predict_test.reset_index()],axis=1).sort_values(['price'])
    #print(product_df_sort)
    r, p_value = pearsonr(product_df_sort['s_count'], product_df_sort['y_predict'])
          
          
    dict_for_model[str(product)+str(store)]['r_'+model_tag+'_test'] = r          
    dict_for_model[str(product)+str(store)]['p_value_'+model_tag+'_test'] = p_value
    
    dict_for_model[str(product)+str(store)][model_tag+'_columns'] = X_train.columns     
        
    return

### Функция проверки качества/удаления плохих моделей из dictionary

In [17]:
# dict_to_check  = name of dictionary with models
# model_tag = 'stacking'
# mark_type = 'wmape' / 'pearsonr' / 'all' = both
# mark_value:   
        #for mark_type = 'wmape' , mark_value = wmape     
        #for mark_type = 'pearsonr' , mark_value = [r , p-value]
        #                                         r - Pearson’s correlation coefficient
        #                                         p - value Two-tailed p-value
        # for mark_type = 'all' , mark_value = [r , p-value, wmape]
        #                                         r - Pearson’s correlation coefficient
        #                                         p - value Two-tailed p-value
        #                                         wmape = wmape
# drop_from_dict = True - удаление из словаря моделей, по которым не прошла проверка качества

def model_control(dict_to_check, model_tag, mark_type, mark_value, drop_from_dict=False):
    
    
    if mark_type == 'wmape':
        table_to_show = pd.DataFrame(dict_to_check).T['wmape_'+model_tag+'_test']
        table_to_show.column = ['wmape_'+model_tag+'_test']
        table_to_show.index.name = 'product_store'
        table_to_show = pd.DataFrame(table_to_show)
        table_to_show['good_estimate'] = (table_to_show['wmape_'+model_tag+'_test'] <= mark_value)    

        list_to_pop = table_to_show[(table_to_show['wmape_'+model_tag+'_test'] <= mark_value) == False ] .index.tolist()
            
            
    elif mark_type == 'pearsonr':
    
        table_to_show = pd.DataFrame(dict_to_check).T[['r_'+model_tag+'_test','p_value_'+model_tag+'_test' ]]
        table_to_show.columns = ['r_'+model_tag+'_test','p_value_'+model_tag+'_test']
        table_to_show.index.name = 'product_store'
        table_to_show = pd.DataFrame(table_to_show)
        table_to_show['good_estimate'] = ((abs(table_to_show['r_'+model_tag+'_test']) >= mark_value[0]) 
                                          & (table_to_show['p_value_'+model_tag+'_test' ] <= mark_value[1])) 

        list_to_pop = table_to_show[((abs(table_to_show['r_'+model_tag+'_test']) >= mark_value[0]) 
                                         & (table_to_show['p_value_'+model_tag+'_test'] <= mark_value[1])) == False].index.tolist()
                                
                                
                                
    elif mark_type == 'all':
    
        table_to_show = pd.DataFrame(dict_to_check).T[['r_'+model_tag+'_test','p_value_'+model_tag+'_test','wmape_'+model_tag+'_test']]
        table_to_show.columns = ['r_'+model_tag+'_test','p_value_'+model_tag+'_test' ,'wmape_'+model_tag+'_test']
        table_to_show.index.name = 'product_store'
        table_to_show = pd.DataFrame(table_to_show)
        table_to_show['good_estimate'] = ((abs(table_to_show['r_'+model_tag+'_test']) >= mark_value[0]) 
                                          & (table_to_show['p_value_'+model_tag+'_test' ] <= mark_value[1]) 
                                          & (table_to_show['wmape_'+model_tag+'_test'] <= mark_value[2])   ) 

    

        list_to_pop = table_to_show[((abs(table_to_show['r_'+model_tag+'_test']) >= mark_value[0]) 
                                         & (table_to_show['p_value_'+model_tag+'_test'] <= mark_value[1]) 
                                         & (table_to_show['wmape_'+model_tag+'_test'] <= mark_value[2])) == False].index.tolist()

                    
    else:
        print('Wrong mark_type. Function model_control() expects mark_type = "wmape" or "pearsonr" or "all".')
              
    
    if drop_from_dict == True:
            [dict_to_check.pop(key) for key in list_to_pop]  
       
    return dict_to_check, table_to_show
                                    

In [18]:
def correlations_draw(dict_for_model, product_store_list, model_tag):
    if len(product_store_list) == 0:
        product_store_list = dict_for_model.keys()
    
    i = 0
    print(len(product_store_list))
    for ps in product_store_list:
        if model_tag in dict_for_model[ps].keys():
            r = dict_for_model[ps]['r_'+model_tag+'_test']
            p_value = dict_for_model[ps]['p_value_'+model_tag+'_test']
            wmape = dict_for_model[ps]['wmape_'+model_tag+'_test']
            y_predict = pd.DataFrame(dict_for_model[ps][model_tag+'_y_predict'])
            y_predict.columns = ['y_predict']
            dt = pd.concat( [pd.concat([dict_for_model[ps]['price_test'],dict_for_model[ps]['s_count_test']],axis=1).reset_index(),
                            y_predict], axis = 1).sort_values(['price']).reset_index()
            #print(dt)
            plt.figure(figsize=(12,4))
            plt.plot(dt['price'],dt['s_count'],label='fact')
            plt.plot(dt['price'],dt['y_predict'], label='predict')
            plt.title(str(ps)+ '. wmape = ' + str(round(wmape,4)) + '. r = ' + str(round(r,4)) + '. p_value = ' + str(p_value))
            plt.legend(loc='upper right', frameon=False)
            plt.xlabel('Price')
            plt.ylabel('Sales')
            #lt.xticks(np.arange(0, len(dt['price'])+1, 2))
            plt.show()
    return    

## Собственно начало основного блока (вызов функций загрузки данных, предобработки)

In [19]:
price_context = load_pc() #load_type='db'
target_plan = load_plan() 

Start loading data from PRICE_FORMATION.sales_price_context
Finish loading data from PRICE_FORMATION.sales_price_context
Start loading data from PRICE_FORMATION.target_product_store_plan
Finish loading data from PRICE_FORMATION.target_product_store_plan


In [20]:
pc_prepared, groupl_cols = data_prepare(price_context)
#pc_prepared - датасет для моделирования, 
#groupl_cols - перечень атрибутов (наименование уровней группировок) не участвующих в моделировании, но необходимых для оптимизации

pc_prepared_train, pc_prepared_test = test_train_split(data_table=pc_prepared, test_proportion=0.2)          

#### ДАЛЕЕ ПОДБОР ГИПЕРПАРАМЕТРОВ И ОБУЧЕНИЕ МОДЕЛИ ДЛЯ КАЖДОГО PRODUCT_ID + STORE_ID

#### Цикл по продуктам

In [21]:
dict_for_model = {}
#from sklearn import model_selection

i=1
for product in pc_prepared['product_id'].unique():
    for store in pc_prepared['store_id'].unique():
        i=i+1
        product_pc_prepared_train = pc_prepared_train[(pc_prepared_train['product_id']==product) & (pc_prepared_train['store_id']==store)]
        product_pc_prepared_test =  pc_prepared_test[(pc_prepared_test['product_id']==product) & (pc_prepared_test['store_id']==store)]
              
        product_X_train, product_y_train = x_y_split(data_table=product_pc_prepared_train, target='s_count')
        product_X_test, product_y_test = x_y_split(data_table=product_pc_prepared_test, target='s_count')
                     
        if len(product_pc_prepared_train)>4 and len(product_pc_prepared_test)>0 and np.sum(np.array(product_pc_prepared_test['s_count']))!=0:
        
            #отбор предикторов 
            features_to_remove = feature_selector(product_X_train, product_y_train, 0.9)
            product_X_train_fs = remove_features(product_X_train, features_to_remove)
            product_X_test_fs = remove_features(product_X_test, features_to_remove)
            product_pc_prepared_train_fs = remove_features(product_pc_prepared_train, features_to_remove)
            product_pc_prepared_test_fs = remove_features(product_pc_prepared_test, features_to_remove)
            
            print(i)
            pc_prepared_train_train, pc_prepared_train_test = test_train_split(data_table=product_pc_prepared_train_fs, test_proportion=0.2)          
            X_train_train, X_train_test = x_y_split(data_table=pc_prepared_train_train, target='s_count')
            y_train_train, y_train_test = x_y_split(data_table=pc_prepared_train_test, target='s_count')

            if len(X_train_train)>4 and len(X_train_test)>0 and np.sum(np.array(product_y_test))!=0 and np.sum(np.array(y_train_test))!=0  :

                dict_for_model[str(product)+str(store)] = {}                
                dict_for_model[str(product)+str(store)]['s_count_test'] = product_y_test
                dict_for_model[str(product)+str(store)]['price_test'] = product_X_test['price']
                
                
                #GradientBoostingRegressor
                ##optuna
                gbr_model_optuna = model_formation(X_train_train, X_train_test, y_train_train, y_train_test,
                                                    model_type='gbr', param_definition='by_optuna')
                
                #Random Forest
                ##optuna
                rf_model_optuna = model_formation(X_train_train, X_train_test, y_train_train, y_train_test,
                                                    model_type='rf', param_definition='by_optuna')  
                
                #Ансамбль моделей  
                stacking_model = sr_stacking((rf_model_optuna,gbr_model_optuna), 
                                              RandomForestRegressor(max_depth=rf_model_optuna.max_depth, 
                                                                    n_estimators=rf_model_optuna.n_estimators,
                                                                    random_state=0))
                stacking_model.fit(product_X_train_fs, product_y_train)
                
                y_wmape_calc_save(product=product, store=store, model=stacking_model, model_tag='stacking', 
                                  X_test=product_X_test_fs, y_test=product_y_test, X_train=product_X_train_fs)
                
               # stacking_modelB = MYMODEL(max_depth=rf_model_optuna.max_depth, 
               #                                                     n_estimators=rf_model_optuna.n_estimators,
               #                                                     random_state=0)
               # stacking_modelB.fit(product_X_train_fs, product_y_train)
               # 
               # y_wmape_calc_save(product=product, store=store, model=stacking_modelB, model_tag='stackingB', 
               #                   X_test=product_X_test_fs, y_test=product_y_test)

Training Gradient Boosting Model



[I 2021-07-16 12:03:47,164] A new study created in memory with name: optuna_GradientBoostingRegressor



79 features with zero importance after one-hot encoding.

24 features required for cumulative importance of 0.90 after one hot encoding.
97 features do not contribute to cumulative importance of 0.90.

2


C:\Users\elena.khotlyannik\Anaconda3\lib\site-packages\optuna\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2021-07-16 12:03:47,535] Trial 5 finished with value: 83.09504369680394 and parameters: {'n_estimators': 35, 'learning_rate': 1.0799818569903883, 'max_depth': 11, 'max_features': 7}. Best is trial 5 with value: 83.09504369680394.
[I 2021-07-16 12:03:47,899] Trial 4 finished with value: 111.77623319841861 and parameters: {'n_estimators': 66, 'learning_rate': 1.6043884814292586, 'max_depth': 11, 'max_features': 16}. Best is trial 5 with value: 83.09504369680394.
[I 2021-07-16 12:03:48,022] Trial 8 finished with value: 120.61669354570432 and parameters: {'n_estimators': 94, 'learning_rate': 1.4873279774085968, 'max_depth': 5, 'max_features': 15}. Best is trial 5 with value: 83.09504369680394.
[I 2021-07-16 12:03:48,043] Trial 9 finishe

[I 2021-07-16 12:03:55,839] A new study created in memory with name: optuna_RandomForest
C:\Users\elena.khotlyannik\Anaconda3\lib\site-packages\optuna\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2021-07-16 12:03:57,463] Trial 6 finished with value: 59.3016863325195 and parameters: {'n_estimators': 105, 'max_depth': 10, 'max_features': 5}. Best is trial 6 with value: 59.3016863325195.
[I 2021-07-16 12:03:58,152] Trial 8 finished with value: 61.41253054172949 and parameters: {'n_estimators': 104, 'max_depth': 5, 'max_features': 8}. Best is trial 6 with value: 59.3016863325195.
[I 2021-07-16 12:03:58,310] Trial 3 finished with value: 59.730237732110155 and parameters: {'n_estimators': 228, 'max_depth': 4, 'max_features': 12}. Best is trial 6 with value: 59.3016863325195.
[I 2021-07-16 12:03:58,787] Trial 7 finished with value: 62.241564752

...


82 features with zero importance after one-hot encoding.

24 features required for cumulative importance of 0.90 after one hot encoding.
97 features do not contribute to cumulative importance of 0.90.

10


[I 2021-07-16 12:09:45,304] Trial 5 finished with value: 106.15010271107016 and parameters: {'n_estimators': 30, 'learning_rate': 1.6087199821695406, 'max_depth': 5, 'max_features': 13}. Best is trial 6 with value: 76.92416350172583.
[I 2021-07-16 12:09:45,367] Trial 7 finished with value: 105.31527178618843 and parameters: {'n_estimators': 19, 'learning_rate': 1.2679649231916956, 'max_depth': 12, 'max_features': 17}. Best is trial 6 with value: 76.92416350172583.
[I 2021-07-16 12:09:45,966] Trial 10 finished with value: 61.65744732667013 and parameters: {'n_estimators': 104, 'learning_rate': 1.29031544819328, 'max_depth': 10, 'max_features': 9}. Best is trial 10 with value: 61.65744732667013.
[I 2021-07-16 12:09:46,049] Trial 0 finished with value: 73.26206118114327 and parameters: {'n_estimators': 401, 'learning_rate': 0.8524115739444709, 'max_depth': 2, 'max_features': 10}. Best is trial 10 with value: 61.65744732667013.
[I 2021-07-16 12:09:46,207] Trial 13 finished with value: 74.0

[I 2021-07-16 12:09:56,849] Trial 3 finished with value: 52.3696472632657 and parameters: {'n_estimators': 296, 'max_depth': 5, 'max_features': 16}. Best is trial 4 with value: 51.78155312448821.
[I 2021-07-16 12:09:57,753] Trial 0 finished with value: 52.640420020422575 and parameters: {'n_estimators': 423, 'max_depth': 4, 'max_features': 19}. Best is trial 4 with value: 51.78155312448821.
[I 2021-07-16 12:09:57,983] Trial 1 finished with value: 50.99801580475954 and parameters: {'n_estimators': 355, 'max_depth': 10, 'max_features': 10}. Best is trial 1 with value: 50.99801580475954.
[I 2021-07-16 12:09:58,576] Trial 8 finished with value: 51.611192320754995 and parameters: {'n_estimators': 194, 'max_depth': 10, 'max_features': 4}. Best is trial 1 with value: 50.99801580475954.
[I 2021-07-16 12:09:59,290] Trial 6 finished with value: 52.278408842707385 and parameters: {'n_estimators': 452, 'max_depth': 5, 'max_features': 16}. Best is trial 1 with value: 50.99801580475954.
[I 2021-07-1

Training Gradient Boosting Model



KeyboardInterrupt: 

In [ ]:
#print(dict_for_model.keys())
dict_for_model, model_control_tab = model_control(dict_to_check=dict_for_model, model_tag='stacking', 
                                                  mark_type = 'pearsonr', mark_value = [0.5,0.05], drop_from_dict=False)
print(model_control_tab)
#print(dict_for_model.keys())

In [ ]:
#print(dict_for_model.keys())
dict_for_model, model_control_tab = model_control(dict_to_check=dict_for_model, model_tag='stacking', 
                                                  mark_type = 'wmape', mark_value = 65, drop_from_dict=False)
print(model_control_tab)
#print(dict_for_model.keys())

In [ ]:
print(dict_for_model.keys())
dict_for_model, model_control_tab = model_control(dict_to_check=dict_for_model, model_tag='stacking', 
                                                  mark_type = 'all', mark_value = [0.5,0.05,65], drop_from_dict=False)
print(model_control_tab)
print(dict_for_model.keys())

In [23]:
dict_for_model

{'55251': {'s_count_test': 1950       0.0
  4985      50.0
  5118      30.0
  5418       5.0
  7610      15.0
            ... 
  256570     0.0
  258523     5.0
  260424    30.0
  265055    10.0
  265227     5.0
  Name: s_count, Length: 144, dtype: float64,
  'price_test': 1950      254.170435
  4985      245.700000
  5118      245.700000
  5418      245.700000
  7610      245.700000
               ...    
  256570    254.170435
  258523    245.700000
  260424    245.700000
  265055    245.700000
  265227    245.700000
  Name: price, Length: 144, dtype: float64,
  'stacking': StackingRegressor(estimators=[('model1',
                                 RandomForestRegressor(max_depth=7,
                                                       max_features=17,
                                                       n_estimators=452,
                                                       random_state=0)),
                                ('model2',
                                 GradientBoosti

In [ ]:
correlations_draw(dict_for_model,[],'stacking')

In [ ]:
# Сделать функцию по сохранению словаря в pickle?
dict_name = 'all_models'
pathname = str("C://Users//elena.khotlyannik//Documents//AA//products_models//") 
filename = str(pathname+dict_name+'.csv') 
pickle.dump(dict_for_model, open(filename, 'wb'))

# NLOpt

In [ ]:
# Сделать функцию по загрузке словаря из pickle?
#загружаю словарь с моделями
dict_name = 'all_models'
pathname = str("C://Users//elena.khotlyannik//Documents//AA//products_models//") 
filename = str(pathname+dict_name+'.csv') 
dict_for_model = pickle.load(open(filename, 'rb'))

In [24]:
#генерю датасет для оптимизации на одну дату
pc_to_optimize = pc_prepared_test[pc_prepared_test['s_date']=='2018-02-22']

In [25]:
#убираю из датасета записи, по которым нет модели в словаре
def create_str(x):
    return str(x[0]) + str(x[1])

def drop_unmodeled_from_dt(dt,dict_for_model):
    dt['product_store'] = list(map(create_str, dt[['product_id','store_id']].values))
    dt['isindictionary'] = dt[['product_store']].isin(dict_for_model.keys())
    dt = dt[dt['isindictionary'] == True].drop(['product_store','isindictionary'],axis=1)
    return dt
    
pc_to_optimize = drop_unmodeled_from_dt(pc_to_optimize, dict_for_model)


<ipython-input-25-ee33b7cc0fd5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['product_store'] = list(map(create_str, dt[['product_id','store_id']].values))
<ipython-input-25-ee33b7cc0fd5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['isindictionary'] = dt[['product_store']].isin(dict_for_model.keys())


In [26]:
dict_for_model.keys()
pc_to_optimize

,s_date,product_id,s_count,group_level0,group_level1,group_level2,group_level3,group_level4,group_level5,group_level6,...,product_category=СЛИВКИ,product_category=СМЕТАНА,product_category=СНЕЖОК,product_category=СЫР %,product_category=СЫРОК ТВОРОЖНЫЙ,product_category=ТВОРОГ,product_category=ТВОРОЖНЫЙ ПРОДУКТ,product_category=ШОКОЛАД,product_subtype=МОЛОЧНЫЕ ПРОДУКТЫ,product_type=ПРОДУКТЫ
33320,2018-02-22,5455,5.0,0_1,1_1,2_5,3_1_1(s2),4_8_1(s2),5_179_1(s2),6_179_16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
66748,2018-02-22,5519,20.0,0_1,1_1,2_6,3_1_1(s2),4_8_1(s2),5_181_1(s2),6_181_20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
100996,2018-02-22,5459,0.0,0_1,1_1,2_5,3_1_1(s2),4_8_1(s2),5_179_1(s2),6_179_20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
105029,2018-02-22,5525,20.0,0_1,1_1,2_6,3_1_1(s2),4_8_1(s2),5_181_1(s2),6_181_20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
196035,2018-02-22,5459,5.0,0_1,1_1,2_5,3_1_1(s1),4_8_1(s1),5_179_1(s1),6_179_20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
216690,2018-02-22,5519,10.0,0_1,1_1,2_6,3_1_1(s1),4_8_1(s1),5_181_1(s1),6_181_20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
230327,2018-02-22,5455,10.0,0_1,1_1,2_5,3_1_1(s1),4_8_1(s1),5_179_1(s1),6_179_16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
232362,2018-02-22,9554,20.0,0_1,1_1,2_11,3_1_2(s1),4_8_2(s1),5_189_1(s1),6_189_32,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
244243,2018-02-22,5525,10.0,0_1,1_1,2_6,3_1_1(s1),4_8_1(s1),5_181_1(s1),6_181_20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [27]:
#функция для предсказания значений продаж s_count для оптимизируемого датасета (по каждому product_id+store_id считается по своей модели) 
def predict_calculation(predict_dt, dict_for_model, model_tag = 'stacking' ):
    for product in predict_dt['product_id'].unique():
        for store in predict_dt['store_id'].unique():            
            if (str(product)+str(store)) in dict_for_model.keys() and model_tag in dict_for_model[str(product)+str(store)].keys():
                model = dict_for_model[str(product)+str(store)][model_tag]
                dt_columns = dict_for_model[str(product)+str(store)][model_tag+'_columns']
             #  print(model)
             #  print(product,store,model_name,model,dt_columns)
                product_store_dt = predict_dt[dt_columns][(predict_dt['product_id']==product) & (predict_dt['store_id']==store)]
                predict_dt2 = model.predict(product_store_dt)
                predict_dt.loc[((predict_dt['product_id']==product)& (predict_dt['store_id']==store) ),'s_count'] = predict_dt2
            #y_predict[product_id == product]
    return predict_dt

In [28]:
#функция для пересчета предикторов (ценового контекста), при подстановке новой цены price_new
def change_context(price_new,dt=pc_to_optimize):
    new_pc = dt.copy()
    new_pc['price'] = price_new
    also_columns = ['store_id','price'] #,'year','month','day']
    table_to_groupby = new_pc[special_cols+also_columns+groupl_cols]
    for level in groupl_cols:    

        new_pc = new_pc.drop(['mean', 'max', 'min'], axis=1, errors = 'ignore')
        agg_prices_table =  table_to_groupby.groupby(['store_id',level])['price'].agg(['mean', 'max', 'min'])
        new_pc = new_pc.join(agg_prices_table, on=['store_id',level], sort=False )
        
        cols_level = [col for col in dt.columns if '_cprice_l'+ str(level)[11:] in col]
        for cols in cols_level:
            if 'max' in cols:
                if  'rel_dev' in cols:
                    new_pc[cols] = (new_pc['price'] - new_pc['max']) /  new_pc['price']
                elif 'abs_dev' in cols:
                    new_pc[cols] = new_pc['price'] - new_pc['max']
            elif 'min' in cols:
                if  'rel_dev' in cols:
                    new_pc[cols] = (new_pc['price'] - new_pc['min']) /  new_pc['price']
                elif 'abs_dev' in cols:
                    new_pc[cols] = new_pc['price'] - new_pc['min']
            elif 'avg' in cols:
                if  'rel_dev' in cols:
                    new_pc[cols] = (new_pc['price'] - new_pc['mean']) /  new_pc['price']
                elif 'abs_dev' in cols:
                    new_pc[cols] = new_pc['price'] - new_pc['mean']
    return new_pc.drop(['mean', 'max', 'min'], axis=1).fillna(0)

In [29]:
#ограничение на минимальный план по каждому 'product_id'+'store_id'. Проверяем сумму amount
min_plan_dt = pc_to_optimize
def vector_min_plan_constrain(result_min,X, grad = None):
    global min_plan_step
    global min_plan_dt
    global dict_for_model
    
    min_plan_dt=change_context(X,min_plan_dt)
    min_plan_dt['amount'] = X*predict_calculation(min_plan_dt, dict_for_model)['s_count']
    
    result_tab_sum=min_plan_dt.set_index(['product_id','store_id']).join(target_plan.set_index(['product_id','store_id']),  sort=False)
    result_tab_sum['diff'] =  result_tab_sum['min_plan'] - result_tab_sum['amount']
    print('min_plan_step',min_plan_step)
    print(result_tab_sum[['price', 's_count', 'amount','min_plan', 'diff']].head())
    min_plan_constrain = result_tab_sum.values
    min_plan_step = min_plan_step + 1
    result_min[:]=min_plan_constrain[:,-1]

In [30]:
#ограничение на максимальный план по каждому 'product_id'+'store_id' . Проверяем сумму amount
max_plan_dt = pc_to_optimize
def vector_max_plan_constrain(result_max, X, grad = None):
    global max_plan_step
    global max_plan_dt
    global dict_for_model
    
    max_plan_dt=change_context(X,max_plan_dt)
    max_plan_dt['amount'] = X*predict_calculation(max_plan_dt, dict_for_model)['s_count']
    
    result_tab_sum=max_plan_dt.set_index(['product_id','store_id']).join(target_plan.set_index(['product_id','store_id']), sort=False)
    result_tab_sum['diff'] =  result_tab_sum['amount'] - result_tab_sum['max_plan']    
    print('max_plan_step',max_plan_step)
    print(result_tab_sum[['price', 's_count', 'amount','max_plan', 'diff']].head())
    max_plan_constrain =  result_tab_sum.values
    max_plan_step = max_plan_step + 1
    result_max[:] = max_plan_constrain[:,-1]

In [31]:
#собственно блок оптимизации
i=1
min_plan_step = 1
max_plan_step = 1

myfunc_dataset = pc_to_optimize.copy()

def myfunc(X, grad = None):  # X = это PRICE 
    global i
    global ff
    global myfunc_dataset
    global dict_for_model
    
    myfunc_dataset=change_context(X,myfunc_dataset)
    sumres = sum(X*predict_calculation(myfunc_dataset, dict_for_model)['s_count'])
    #print(X)
    #print(i, X,round(sumres))
    i = i+1
    return sumres

opt = nlopt.opt(nlopt.LN_AUGLAG, len(myfunc_dataset))
#nlopt.set_local_optimizer
opt.set_max_objective(myfunc)

lower_bounds = pc_to_optimize['price']*0.8 
upper_bounds = pc_to_optimize['price']*1.2 
opt.set_xtol_abs(1)
opt.set_ftol_abs(1000)
opt.set_lower_bounds(lower_bounds)
opt.set_upper_bounds(upper_bounds)

opt.add_inequality_mconstraint(vector_min_plan_constrain, [1]* len(myfunc_dataset))
opt.add_inequality_mconstraint(vector_max_plan_constrain, [1]* len(myfunc_dataset))


#opt.set_xtol_rel(1e-1)
#opt.set_ftol_rel(1e-1)

opt.set_maxeval(500)

X = opt.optimize(pc_to_optimize['price'])
maxf = opt.last_optimum_value()


print("optimum at ",  X)
print("maximum value = ", maxf)
print("result code = ", opt.last_optimize_result())

min_plan_step 1
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          82.188889  18.161561  1492.678548  628.618000   
5519       2         245.700000  26.886166  6605.931100  385.028024   
5459       2         102.493934   3.693181   378.528608  586.981538   
5525       2         284.700000  26.247275  7472.599190  465.760329   
5459       1         104.698925   9.011340   943.477638  689.910842   

                            diff  
product_id store_id               
5455       2         -864.060548  
5519       2        -6220.903076  
5459       2          208.452931  
5525       2        -7006.838860  
5459       1         -253.566795  
max_plan_step 1
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          82.188889  18.161561  1492.678548   942.927000   
5519     

max_plan_step 6
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          90.407778  17.856883  1614.401148   942.927000   
5519       2         245.700000  26.627052  6542.266621   577.542035   
5459       2         112.743328   9.844975  1109.955261   880.472308   
5525       2         313.170000  27.185655  8513.731471   698.640494   
5459       1         104.698925   9.011340   943.477638  1034.866263   

                            diff  
product_id store_id               
5455       2          671.474148  
5519       2         5964.724586  
5459       2          229.482953  
5525       2         7815.090977  
5459       1          -91.388626  
min_plan_step 7
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          90.407778  17.856883  1614.401148  628.618000   
5519 

min_plan_step 12
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          90.534303  17.554835  1589.314747  628.618000   
5519       2         241.855926  26.783238  6477.684941  385.028024   
5459       2         114.308241   9.856819  1126.715665  586.981538   
5525       2         274.604522  26.279223  7216.393351  465.760329   
5459       1         115.986097   8.695932  1008.607242  689.910842   

                            diff  
product_id store_id               
5455       2         -960.696747  
5519       2        -6092.656918  
5459       2         -539.734127  
5525       2        -6750.633022  
5459       1         -318.696399  
max_plan_step 12
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          90.534303  17.554835  1589.314747   942.927000   
5519   

max_plan_step 17
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          92.353805  17.572303  1622.869096   942.927000   
5519       2         253.367215  14.704293  3725.585782   577.542035   
5459       2         116.056112   9.856819  1143.944111   880.472308   
5525       2         288.268783  26.279223  7575.479504   698.640494   
5459       1         116.231188   8.695932  1010.738544  1034.866263   

                            diff  
product_id store_id               
5455       2          679.942096  
5519       2         3148.043747  
5459       2          263.471804  
5525       2         6876.839010  
5459       1          -24.127719  
min_plan_step 18
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          92.368884  18.489873  1707.888953  628.618000   
551

min_plan_step 23
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          92.344559  17.837325  1647.179916  628.618000   
5519       2         253.553341  27.182725  6892.270740  385.028024   
5459       2         114.091065   9.819465  1120.313216  586.981538   
5525       2         287.612503  26.279223  7558.232969  465.760329   
5459       1         115.823739   8.695932  1007.195394  689.910842   

                            diff  
product_id store_id               
5455       2        -1018.561916  
5519       2        -6507.242716  
5459       2         -533.331678  
5525       2        -7092.472640  
5459       1         -317.284552  
max_plan_step 23
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          92.344559  17.837325  1647.179916   942.927000   
5519   

...

max_plan_step 226
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          95.499739  17.651527  1685.716235   942.927000   
5519       2         196.560000  26.379162  5185.088121   577.542035   
5459       2         121.893301   9.772064  1191.149095   880.472308   
5525       2         227.760000  24.970601  5687.304162   698.640494   
5459       1         119.262211   8.495524  1013.194980  1034.866263   

                            diff  
product_id store_id               
5455       2          742.789235  
5519       2         4607.546085  
5459       2          310.676787  
5525       2         4988.663668  
5459       1          -21.671283  
min_plan_step 227
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          95.341359  17.651527  1682.920586  628.618000   
5

min_plan_step 232
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          95.499739  17.651527  1685.716235  628.618000   
5519       2         196.560000  26.379162  5185.088121  385.028024   
5459       2         121.893301   9.772064  1191.149095  586.981538   
5525       2         227.760000  24.970601  5687.304162  465.760329   
5459       1         119.262211   8.495524  1013.194980  689.910842   

                            diff  
product_id store_id               
5455       2        -1057.098235  
5519       2        -4800.060097  
5459       2         -604.167556  
5525       2        -5221.543832  
5459       1         -323.284138  
max_plan_step 232
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          95.499739  17.651527  1685.716235   942.927000   
5519 

max_plan_step 237
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          95.401873  17.708637  1689.437104   942.927000   
5519       2         200.801294  26.352567  5291.629630   577.542035   
5459       2         122.615498   9.762434  1197.025692   880.472308   
5525       2         228.345017  25.004617  5709.679621   698.640494   
5459       1         119.073177   8.674473  1032.897069  1034.866263   

                            diff  
product_id store_id               
5455       2          746.510104  
5519       2         4714.087595  
5459       2          316.553384  
5525       2         5011.039127  
5459       1           -1.969194  
min_plan_step 238
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          95.499739  17.651527  1685.716235  628.618000   
5

min_plan_step 243
                          price    s_count       amount    min_plan  \
product_id store_id                                                   
5455       2          96.164473  17.666131  1698.854196  628.618000   
5519       2         221.130000  24.306811  5374.965185  385.028024   
5459       2         120.812401   9.933350  1200.071839  586.981538   
5525       2         227.760000  25.232909  5747.047384  465.760329   
5459       1         118.969188   8.495524  1010.705592  689.910842   

                            diff  
product_id store_id               
5455       2        -1070.236196  
5519       2        -4989.937161  
5459       2         -613.090301  
5525       2        -5281.287055  
5459       1         -320.794749  
max_plan_step 243
                          price    s_count       amount     max_plan  \
product_id store_id                                                    
5455       2          96.164473  17.666131  1698.854196   942.927000   
5519 

KeyboardInterrupt: 

In [ ]:
X

In [ ]:
pc_to_optimize_result = pd.DataFrame(np.hstack((pc_to_optimize[['s_date','product_id','store_id','price']],pd.DataFrame(X))))#.columns(['s_date','product_id','store_id','price','opt_price'])
pc_to_optimize_result.columns = ['s_date','product_id','store_id','price','opt_price']
pc_to_optimize_result